In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
from implementations import *

In [3]:
DATA_TRAIN_PATH = '../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [22]:
M = tX.shape[1] # number of features

In [15]:
train_x, train_y, test_x, test_y = split_data(tX, y, 0.5) # split into train and test

In [18]:
# group data by PRI_jet_num (col_index = 22)
def separate_data(X, y, val):
    mask = X[:, 22] == val
    return X[mask], y[mask]

In [37]:
x0, y0 = separate_data(train_x, train_y, 0)
x1, y1 = separate_data(train_x, train_y, 1)
x2, y2 = separate_data(train_x, train_y, 2)
x3, y3 = separate_data(train_x, train_y, 3)

In [54]:
# compute weights for columns we selected
def regress(X, y, mask):
    cleaned = np.take(X, mask, axis=1)
    w,_ = least_squares(y, cleaned)
    return w

In [55]:
# put 0 weights where the removed columns are, so we can simply multiply with test set without removing columns again
def rescale(w, mask, M=M):
    out = np.zeros((M,))
    for e,i in enumerate(mask):
        out[i] = w[e]
    return out

In [89]:
# choose relevant columns for each group
mask0 = [1,2,5,12,13,14,15,16]
mask1 = [1,2,3,4]
mask2 = [2]
mask3 = [24]

In [90]:
w0 = rescale(regress(x0,y0,mask0), mask0)
w1 = rescale(regress(x0,y0,mask1), mask1)
w2 = rescale(regress(x0,y0,mask2), mask2)
w3 = rescale(regress(x0,y0,mask3), mask3)
ws = [w0,w1,w2,w3]

In [91]:
def predict(ws, X):
    labels = []
    for sample in X:
        labels.append(sample @ ws[int(sample[22])])
    labels = np.array(labels)
    labels[labels >= 0] = 1
    labels[labels < 0] = -1
    return labels

In [92]:
predictions = predict(ws, test_x)

In [93]:
print('Accuracy on',len(test_y),'test samples is',np.sum(predictions==test_y)/len(test_y))

Accuracy on 125000 test samples is 0.661248


## Old stuff

In [344]:
train_x, train_y, test_x, test_y = split_data(X, y, 0.5)
weights, loss = least_squares(train_y, train_x)
# Accuracy
y_check = predict_labels(weights, test_x)
print('Accuracy on',len(test_y),'test samples is',np.sum(y_check==test_y)/len(y_check))

Accuracy on 125000 test samples is 0.74408


## Generate predictions and save ouput in csv format for submission:

In [75]:
DATA_TEST_PATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [76]:
OUTPUT_PATH = '../output/predictions_0.csv'
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)